In [ ]:
import os
import re
import numpy as np
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor, as_completed
from MMGenerateFunc import MMGenerateFunctions, MMPlot


def process_bin_file(bin_path, wordIndex, fileIndex, personIndex, numTx, numRx):
    # 这里调用你的处理函数

    # 获取bin文件数据
    data_raw = MMGenerateFunctions.getRawData_multiTx(
        bin_path, numSamples, numChirps, numFrames, numTx, numRx
    )  # numRx, numFrames, numSamples

    data_1dfft_dyn_all = np.zeros(data_raw[0].shape, dtype=complex)

    for tx in range(numTx):
        for rx in range(numRx):
            # 1D-FFT
            data_1dfft = MMGenerateFunctions.rangeFFT(
                data_raw[tx][rx], numFrames, numSamples
            )
            # 去除静态杂波
            data_1dfft_dyn_all[rx, :, :] = MMGenerateFunctions.removeStatic(
                data_1dfft, numFrames, numSamples
            )
            # data_1dfft_dyn = data_1dfft

        numAngles = 90
        # 3D-FFT
        data_3dfft = MMGenerateFunctions.angleFFT(
            data_1dfft_dyn_all, numFrames, numSamples, numAngles
        )

        range_index, angle_index = MMGenerateFunctions.getRangeAngleIndex(data_3dfft)

        # 提取相位并解缠绕
        range_angle, angle_diff = MMGenerateFunctions.getUnwrapAngle(
            data_3dfft[:, range_index, angle_index]
        )

        # 对相位差进行带通滤波
        # angle_diff_filtered = MMGenerateFunctions.filteredAngle(angle_diff, lowcut, highcut, fs)
        angle_diff_filtered = angle_diff

        # 计算STFT
        f, t, Zxx = MMGenerateFunctions.getSTFT(
            angle_diff_filtered, window_size, overlap, fs
        )

        # 绘制结果
        MMPlot.saveSTFT(t, f, Zxx, save_path, wordIndex, fileIndex, personIndex, tx)


# 雷达参数配置
numSamples = 80
numChirps = 1
numFrames = 64000
numTx = 3
numRx = 4

# 滤波器参数
lowcut = 90  # 下限频率 (Hz)
highcut = 300.0  # 上限频率 (Hz)
fs = 1600.0  # 采样频率 (Hz)

# STFT参数
window_size = 96
overlap = 0.7

# 路径和文件夹列表
base_path = r"/home/mambauser/MMCode/data/rawData"
save_path = r"/home/mambauser/MMCode/data/processed3D"

personIndex = 0


# 使用进程池并行处理文件
def main():
    bin_files = [f for f in os.listdir(base_path) if f.endswith(".bin")]
    total_files = len(bin_files)
    tasks = []

    for file_name in tqdm(bin_files):
        # 使用正则表达式匹配文件名
        match = re.match(rf"(\d+)_(\d+)_Raw_?0\.bin", file_name)
        if match:
            # 提取字母和数字
            letter_part = int(match.group(1))
            number_part = int(match.group(2))

            bin_path = os.path.join(base_path, file_name)

            letterIndex = letter_part
            fileIndex = number_part

            process_bin_file(
                bin_path, letterIndex, fileIndex, personIndex, numTx, numRx
            )


if __name__ == "__main__":
    main()